# SGFA qMAP-PD Analysis on GPU

**Updated for comprehensive experimental framework**

This notebook provides GPU-accelerated analysis of Parkinson's disease neuroimaging data using Sparse Group Factor Analysis (SGFA). The codebase now includes a comprehensive experimental validation framework.

## Key Updates:
- **Experimental Framework**: Comprehensive validation suite
- **Modular Analysis**: Data validation, method comparison, clinical validation
- **Performance Optimization**: Memory management and streaming
- **Cross-Validation**: Advanced CV strategies and nested validation
- **Brain Mapping**: Factor-to-neuroimaging visualization


## Environment Setup

Creates Python 3.10 kernel with JAX CUDA support and mounts Google Drive for persistent results.

In [ ]:
import os, subprocess, sys, stat, textwrap, pathlib, shutil, json

REPO = "/content/sgfa_qmap-pd"
REPO_URL = "https://github.com/meeramads/sgfa_qmap-pd.git"  # Update with your repo URL

In [ ]:
# Clone the repository
os.chdir("/content")
subprocess.run(["rm", "-rf", "sgfa_qmap-pd"])
subprocess.check_call(["git", "clone", REPO_URL])
os.chdir(REPO)

print(f"Repository cloned to: {REPO}")
print(f"Current directory: {os.getcwd()}")

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')

In [ ]:
# Set up persistent results directory
drive_path = "/content/drive/MyDrive/sgfa_experiments"

if os.path.exists(drive_path):
    if os.path.isdir(drive_path):
        print(f"Found existing experiments directory with {len(os.listdir(drive_path))} items")
    else:
        print("Found file named 'sgfa_experiments' - backing up and creating directory")
        shutil.move(drive_path, f"{drive_path}_backup")
        os.makedirs(drive_path)
else:
    print("Creating new experiments directory")
    os.makedirs(drive_path)

# Create symlinks for results
for link_name in ["experiment_results", "results"]:
    link_path = f"../{link_name}"
    if os.path.exists(link_path):
        if os.path.islink(link_path):
            os.unlink(link_path)
        else:
            shutil.rmtree(link_path)
    os.symlink(drive_path, link_path)

print(f"Results will be saved to: {drive_path}")

## Python and CUDA Environment

**Must be connected to a GPU runtime.** Sets up Python 3.10 with JAX CUDA support.

In [ ]:
# Install Python 3.10
subprocess.check_call(["wget", "-q", "https://github.com/korakot/kora/releases/download/v0.10/py310.sh"])
subprocess.check_call(["bash", "./py310.sh", "-b", "-f", "-p", "/usr/local"])
subprocess.check_call(["python3.10", "-V"])

In [ ]:
# Install dependencies
subprocess.check_call(["python3.10", "-m", "pip", "install", "-U", "pip"])
subprocess.check_call(["python3.10", "-m", "pip", "install", "-r", "requirements.txt"])

In [ ]:
# Set up JAX with CUDA support
subprocess.run(["python3.10", "-m", "pip", "uninstall", "-y", "jax", "jaxlib"])
subprocess.check_call([
    "python3.10", "-m", "pip", "install", "-U",
    "jax[cuda12_pip]==0.4.20", "-f", "https://storage.googleapis.com/jax-releases/jax_cuda_releases.html"
])
subprocess.check_call(["python3.10", "-m", "pip", "install", "numpyro==0.13.2"])

In [ ]:
# Install NVIDIA CUDA libraries
subprocess.check_call(["python3.10", "-m", "pip", "install", "-q",
    "nvidia-cudnn-cu12>=8.9,<9",
    "nvidia-cublas-cu12>=12.2",
    "nvidia-cuda-runtime-cu12>=12.2",
    "nvidia-cusolver-cu12>=11.4",
    "nvidia-cusparse-cu12>=12.1",
    "nvidia-cufft-cu12>=11.0",
    "nvidia-cuda-cupti-cu12>=12.2",
    "nvidia-nvjitlink-cu12>=12.2",
    "nvidia-nccl-cu12>=2.18",
])

In [ ]:
# Create CUDA-enabled Python launcher
py310_site = subprocess.check_output(
    ["python3.10", "-c", "import site; print(site.getsitepackages()[0])"],
    text=True
).strip()

subdirs = ["cudnn/lib", "cublas/lib", "cufft/lib", "cusolver/lib", "cusparse/lib",
           "cuda_runtime/lib", "cuda_cupti/lib", "nvjitlink/lib", "nccl/lib"]
lib_paths = [os.path.join(py310_site, "nvidia", d) for d in subdirs]
lib_paths = [p for p in lib_paths if os.path.isdir(p)]
LD = ":".join(lib_paths)

wrapper = "/usr/local/bin/py310cuda"
pathlib.Path(wrapper).write_text(textwrap.dedent(f"""\
#!/bin/bash
export LD_LIBRARY_PATH="{LD}:$LD_LIBRARY_PATH"
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_MEM_FRACTION=0.75
export JAX_PLATFORM_NAME=gpu
export JAX_ENABLE_X64=true
exec python3.10 "$@"
"""))
os.chmod(wrapper, os.stat(wrapper).st_mode | stat.S_IEXEC)

print("CUDA launcher created successfully")

In [ ]:
# Verify JAX GPU setup
!py310cuda -c "import jax; print('Platform:', jax.lib.xla_bridge.get_backend().platform); print('Devices:', jax.devices())"

---

# Experimental Framework Usage

The codebase now includes a comprehensive experimental framework. You can either use:

1. **Legacy Analysis**: `run_analysis.py` for direct model runs
2. **Experimental Framework**: Structured experiments with validation and comparison

## Quick Status Check

In [ ]:
!git pull
!ls -la

In [ ]:
# Quick system check
print("=== System Status ===")
!py310cuda -c "import jax; print(f'JAX backend: {jax.lib.xla_bridge.get_backend().platform}')"
!py310cuda -c "import jax; print(f'JAX devices: {jax.devices()}')"
!py310cuda -c "import numpy as np; print(f'NumPy version: {np.__version__}')"
!py310cuda -c "import numpyro; print(f'NumPyro version: {numpyro.__version__}')"

print("\n=== Repository Structure ===")
!find . -maxdepth 2 -type d | head -15

print("\n=== Available Datasets ===")
!ls -la qMAP-PD_data/ 2>/dev/null || echo "qMAP-PD data directory not found"

print("\n=== Testing Framework Imports ===")

# Create import test script
import_test_script = '''
import sys
import os
sys.path.insert(0, '.')

FRAMEWORK_AVAILABLE = True
error_messages = []

# Test core module imports
try:
    from core import run_analysis, utils, get_data, visualization
    print("Core modules imported")
except Exception as e:
    print(f"Core modules failed: {e}")
    error_messages.append(f"Core: {e}")

# Test experimental framework imports
try:
    from experiments.framework import ExperimentConfig, ExperimentFramework
    print("ExperimentFramework imported")
except Exception as e:
    print(f"ExperimentFramework failed: {e}")
    error_messages.append(f"ExperimentFramework: {e}")
    FRAMEWORK_AVAILABLE = False

try:
    from experiments.data_validation import DataValidationExperiments
    print("DataValidationExperiments imported")
except Exception as e:
    print(f"DataValidationExperiments failed: {e}")
    error_messages.append(f"DataValidationExperiments: {e}")
    FRAMEWORK_AVAILABLE = False

# Test legacy analysis imports
try:
    from analysis.cross_validation import CVRunner
    print("Legacy CVRunner imported")
except Exception as e:
    print(f"Legacy CVRunner failed: {e}")
    error_messages.append(f"CVRunner: {e}")

# Test basic model imports
try:
    from models import create_model
    print("Model factory imported")
except Exception as e:
    print(f"Model factory failed: {e}")

# Save detailed status
status_info = {
    "framework_available": FRAMEWORK_AVAILABLE,
    "errors": error_messages,
    "recommendation": "experimental" if FRAMEWORK_AVAILABLE else "legacy"
}

with open("framework_status.txt", "w") as f:
    f.write("available" if FRAMEWORK_AVAILABLE else "unavailable")

with open("import_status.txt", "w") as f:
    f.write(str(status_info))

if FRAMEWORK_AVAILABLE:
    print("\\nExperimental framework ready!")
    print("   → You can use all experimental modules")
else:
    print("\\nExperimental framework has issues")
    print("   → Use legacy analysis methods instead")
    print("   → core/run_analysis.py should still work")

print(f"\\nStatus saved to framework_status.txt")
'''

# Write and execute the test
with open("test_imports.py", "w") as f:
    f.write(import_test_script)

!py310cuda test_imports.py

# Experimental Framework

## Module 1: Data Validation Experiments

Comprehensive data quality assessment and preprocessing validation.

In [ ]:
# Import Troubleshooting & Fallback Guide
print("=== Import Status Check ===")

# Check framework status
framework_available = False
try:
    with open("framework_status.txt", "r") as f:
        status = f.read().strip()
        framework_available = (status == "available")
        print(f"Framework status: {status}")
except FileNotFoundError:
    print("Framework status: not tested yet - run the system check cell above")

if framework_available:
    print(" Experimental framework is available!")
    print("  You can run the experimental framework cells below")
    print("   Advanced validation, comparison, and clinical experiments")
else:
    print("  Experimental framework has import issues")
    print("   SOLUTION: Use legacy analysis instead")
    print("   run_analysis.py should work fine")
    print("   All core SGFA functionality available")

print("\n=== Recommended Workflow ===")
if framework_available:
    print("1. Run experimental framework cells (data validation, method comparison, etc.)")
    print("2. OR run legacy analysis for reliable results")
else:
    print("1. Skip experimental framework cells (they will show warnings)")
    print("2. Use legacy analysis cells - these are very reliable")
    print("3. Start with 'Quick smoke test' cell to verify everything works")

print("\n=== Legacy Analysis Available ===")
print("• Basic qMAP-PD analysis")  
print("• Preprocessing pipeline tests")
print("• Cross-validation experiments")
print("• Comprehensive qMAP-PD analysis")
print("• All core SGFA functionality")

print("\nBoth approaches provide quality results!")

In [ ]:
# Data Validation Experiments - with enhanced error handling
import os

# Check if experimental framework is available
framework_available = True
if os.path.exists('framework_status.txt'):
    with open('framework_status.txt', 'r') as f:
        framework_available = f.read().strip() == 'available'

if framework_available:
    print("🚀 Running experimental framework data validation...")
    
    data_validation_script = """
import sys
sys.path.append('.')
import traceback

try:
    from experiments.data_validation import DataValidationExperiments
    from experiments.framework import ExperimentConfig, ExperimentFramework
    from pathlib import Path

    # Create experiment framework with output directory
    framework = ExperimentFramework(
        base_output_dir=Path("../experiment_results")
    )

    # Configure experiment with proper data directory
    config = ExperimentConfig(
        experiment_name="data_validation_colab",
        description="Comprehensive data validation on qMAP-PD dataset",
        dataset="qmap_pd",
        data_dir="qMAP-PD_data"
    )

    # Create validator with framework
    validator = DataValidationExperiments(framework)

    print("Running data quality assessment...")
    try:
        quality_result = validator.run_data_quality_assessment(config)
        print(f"Data quality assessment completed: {quality_result.status}")
        
        # Try to access results
        if hasattr(quality_result, 'results') and quality_result.results:
            print(f" Generated results with {len(quality_result.results)} metrics")
        if hasattr(quality_result, 'artifacts') and quality_result.artifacts:
            print(f" Created {len(quality_result.artifacts)} artifacts")
            
    except Exception as e:
        print(f" Data quality assessment failed with detailed error:")
        print(f"Error type: {type(e).__name__}")
        print(f"Error message: {str(e)}")
        print("Full traceback:")
        traceback.print_exc()

    # Test simpler preprocessing comparison
    print("\\nRunning preprocessing comparison...")
    try:
        preprocessing_result = validator.run_preprocessing_comparison(config)
        print(f"Preprocessing comparison completed: {preprocessing_result.status}")
        
        if hasattr(preprocessing_result, 'results') and preprocessing_result.results:
            print(f" Generated results with {len(preprocessing_result.results)} comparisons")
            
    except Exception as e:
        print(f" Preprocessing comparison failed:")
        print(f"Error: {str(e)}")
        traceback.print_exc()

    print("\\nData validation experiments completed!")
    print(" Check ../experiment_results/ for any generated plots and reports")
    
    # Test direct data loading to verify the issue
    print("\\n=== Testing Direct Data Loading ===")
    try:
        from data.qmap_pd import load_qmap_pd
        
        print("Loading qMAP-PD data directly...")
        test_data = load_qmap_pd(
            data_dir="qMAP-PD_data",
            enable_advanced_preprocessing=False
        )
        
        print(f" Direct data loading successful!")
        print(f"   - Data keys: {list(test_data.keys())}")
        print(f"   - X_list length: {len(test_data['X_list'])}")
        print(f"   - Shapes: {[X.shape for X in test_data['X_list']]}")
        print(f"   - View names: {test_data.get('view_names', [])}")
        
    except Exception as e:
        print(f" Direct data loading failed: {e}")
        traceback.print_exc()
    
except ImportError as e:
    print(f" Cannot run experimental framework: {e}")
    print(" Try the legacy analysis cells instead")
except Exception as e:
    print(f" Experiment execution failed: {e}")
    traceback.print_exc()
"""

    # Write and run the script
    with open("data_validation_run.py", "w") as f:
        f.write(data_validation_script)

    !py310cuda data_validation_run.py
    
else:
    print("  Experimental framework not available.")
    print(" Use legacy analysis instead:")
    print("   • Basic qMAP-PD analysis")
    print("   • Preprocessing pipeline tests") 
    print("   • Cross-validation experiments")
    print("\nRun the legacy analysis cells below for reliable results.")

## Module 2: Method Comparison Experiments

Compare different SGFA variants and traditional methods.

In [ ]:
# Method Comparison Experiments - with correct API usage
import os

framework_available = True
if os.path.exists('framework_status.txt'):
    with open('framework_status.txt', 'r') as f:
        framework_available = f.read().strip() == 'available'

if framework_available:
    print(" Running method comparison experiments...")
    
    method_comparison_script = """
import sys
sys.path.append('.')

try:
    from experiments.method_comparison import MethodComparisonExperiments
    from experiments.framework import ExperimentConfig, ExperimentFramework
    from pathlib import Path

    # Create experiment framework
    framework = ExperimentFramework(
        base_output_dir=Path("../experiment_results")
    )

    # Configure experiment with Colab-friendly parameters
    config = ExperimentConfig(
        experiment_name="method_comparison_colab",
        description="SGFA variants and traditional method comparison",
        dataset="qmap_pd",
        # Reduced parameters for Colab
        K_values=[8, 12],
        num_samples=500,
        num_warmup=250,
        num_chains=2,
        num_runs_per_config=1
    )

    # Create comparator
    comparator = MethodComparisonExperiments(framework)

    print("Running GFA variant comparison...")
    try:
        variant_result = comparator.run_gfa_variant_comparison(config)
        print(f"GFA variant comparison completed: {variant_result.status}")
    except Exception as e:
        print(f"GFA variant comparison failed: {e}")

    print("\\nRunning traditional method comparison...")
    try:
        traditional_result = comparator.run_traditional_method_comparison(config)
        print(f"Traditional method comparison completed: {traditional_result.status}")
    except Exception as e:
        print(f"Traditional method comparison failed: {e}")

    print("\\nMethod comparison experiments completed!")
    
except ImportError as e:
    print(f" Cannot run experimental framework: {e}")
    print(" Try the legacy analysis cells instead")
except Exception as e:
    print(f" Experiment execution failed: {e}")
"""

    with open("method_comparison_run.py", "w") as f:
        f.write(method_comparison_script)

    !py310cuda method_comparison_run.py
    
else:
    print("  Experimental framework not available - using legacy analysis instead")

## Module 3: Performance Benchmarks

Scalability and efficiency testing optimized for GPU.

In [ ]:
# Performance Benchmarks - with correct API usage  
import os

framework_available = True
if os.path.exists('framework_status.txt'):
    with open('framework_status.txt', 'r') as f:
        framework_available = f.read().strip() == 'available'

if framework_available:
    print(" Running performance benchmarks...")
    
    performance_script = """
import sys
sys.path.append('.')

try:
    from experiments.performance_benchmarks import PerformanceBenchmarkExperiments
    from experiments.framework import ExperimentConfig, ExperimentFramework
    from pathlib import Path

    # Create experiment framework
    framework = ExperimentFramework(
        base_output_dir=Path("../experiment_results")
    )

    # Configure for GPU performance testing
    config = ExperimentConfig(
        experiment_name="performance_benchmarks_gpu",
        description="GPU performance benchmarking for SGFA",
        dataset="synthetic",  # Use synthetic for controlled benchmarking
        K_values=[5, 10, 15],
        num_samples=400,
        num_warmup=200,
        num_chains=2,
        num_runs_per_config=1
    )

    # Create benchmarker
    benchmarker = PerformanceBenchmarkExperiments(framework)

    print("Running scalability benchmarks...")
    try:
        scalability_result = benchmarker.run_scalability_benchmarks(config)
        print(f"Scalability benchmarks completed: {scalability_result.status}")
    except Exception as e:
        print(f"Scalability benchmarks failed: {e}")

    print("\\nRunning memory benchmarks...")
    try:
        memory_result = benchmarker.run_memory_benchmarks(config)
        print(f"Memory benchmarks completed: {memory_result.status}")
    except Exception as e:
        print(f"Memory benchmarks failed: {e}")

    print("\\nPerformance benchmarking completed!")
    
except ImportError as e:
    print(f" Cannot run experimental framework: {e}")
    print(" Try the legacy analysis cells instead")
except Exception as e:
    print(f" Experiment execution failed: {e}")
"""

    with open("performance_run.py", "w") as f:
        f.write(performance_script)

    !py310cuda performance_run.py
    
else:
    print("  Experimental framework not available - using legacy analysis instead")

## Module 4: Clinical Validation

Parkinson's disease subtype classification and clinical validation.

In [ ]:
# Clinical Validation - with correct API usage
import os

framework_available = True
if os.path.exists('framework_status.txt'):
    with open('framework_status.txt', 'r') as f:
        framework_available = f.read().strip() == 'available'

if framework_available:
    print(" Running clinical validation...")
    
    clinical_script = """
import sys
sys.path.append('.')

try:
    from experiments.clinical_validation import ClinicalValidationExperiments
    from experiments.framework import ExperimentConfig, ExperimentFramework
    from pathlib import Path

    # Create experiment framework
    framework = ExperimentFramework(
        base_output_dir=Path("../experiment_results")
    )

    # Configure clinical validation
    config = ExperimentConfig(
        experiment_name="clinical_validation_pd",
        description="Parkinson's disease clinical validation",
        dataset="qmap_pd",
        K_values=[12, 18],
        num_samples=1000,
        num_warmup=500,
        num_chains=2,
        num_runs_per_config=1
    )

    # Create clinical validator
    clinical_validator = ClinicalValidationExperiments(framework)

    print("Running PD subtype classification...")
    try:
        subtype_result = clinical_validator.run_pd_subtype_classification(config)
        print(f"PD subtype classification completed: {subtype_result.status}")
    except Exception as e:
        print(f"PD subtype classification failed: {e}")

    print("\\nRunning biomarker discovery...")
    try:
        biomarker_result = clinical_validator.run_biomarker_discovery(config)
        print(f"Biomarker discovery completed: {biomarker_result.status}")
    except Exception as e:
        print(f"Biomarker discovery failed: {e}")

    print("\\nClinical validation experiments completed!")
    
except ImportError as e:
    print(f" Cannot run experimental framework: {e}")
    print(" Try the legacy analysis cells instead")
except Exception as e:
    print(f" Experiment execution failed: {e}")
    print(" This may be due to missing clinical data or other dependencies")
"""

    with open("clinical_validation_run.py", "w") as f:
        f.write(clinical_script)

    !py310cuda clinical_validation_run.py
    
else:
    print("  Experimental framework not available - using legacy analysis instead")

# Legacy Analysis (run_analysis.py)

Direct model runs using the original interface.

In [ ]:
# Quick smoke test - This should always work
print("=== Running Legacy Analysis Smoke Test ===")
print("If experimental framework failed, this provides a reliable fallback")

!py310cuda core/run_analysis.py \
  --dataset synthetic \
  --K 5 \
  --num-samples 200 \
  --num-warmup 100 \
  --num-chains 2 \
  --num-runs 1 \
  --device gpu \
  --seed 42

print("\n Legacy analysis smoke test completed!")
print("If this worked, you can run all the other legacy analysis cells below.")

In [ ]:
# Basic qMAP-PD analysis
print("Running basic qMAP-PD analysis...")
!py310cuda core/run_analysis.py \
  --dataset qmap_pd \
  --K 10 \
  --num-samples 800 \
  --num-warmup 400 \
  --num-chains 2 \
  --num-runs 2 \
  --device gpu \
  --seed 42

In [ ]:
# Test preprocessing pipeline
print("Testing preprocessing pipeline...")
!py310cuda core/run_analysis.py \
  --dataset qmap_pd \
  --K 8 \
  --num-samples 500 \
  --num-warmup 250 \
  --num-chains 2 \
  --enable_preprocessing \
  --feature_selection variance \
  --n_top_features 200 \
  --device gpu \
  --seed 42

In [ ]:
# Cross-validation test
print("Testing cross-validation...")
!py310cuda core/run_analysis.py \
  --dataset synthetic \
  --K 5 \
  --num-samples 400 \
  --num-warmup 200 \
  --num-chains 2 \
  --cv_only \
  --cv_folds 3 \
  --device gpu \
  --seed 42

# Advanced Analysis Pipelines

Computationally intensive analyses - run selectively based on available time/resources.

In [ ]:
# Comprehensive qMAP-PD analysis
proceed = input("Run comprehensive qMAP-PD analysis? (15-20 minutes) [y/N]: ")
if proceed.lower() in ['y', 'yes']:
    !py310cuda core/run_analysis.py \
      --dataset qmap_pd \
      --K 20 \
      --num-samples 2000 \
      --num-warmup 1000 \
      --num-chains 3 \
      --num-runs 3 \
      --percW 33 \
      --enable_preprocessing \
      --feature_selection combined \
      --n_top_features 400 \
      --device gpu \
      --seed 42
    print("\nComprehensive analysis completed!")
else:
    print("Skipped comprehensive analysis.")

In [ ]:
# Full cross-validation analysis
proceed = input("Run full cross-validation analysis? (20-25 minutes) [y/N]: ")
if proceed.lower() in ['y', 'yes']:
    !py310cuda core/run_analysis.py \
      --dataset qmap_pd \
      --K 15 \
      --num-samples 1200 \
      --num-warmup 600 \
      --num-chains 3 \
      --run_cv \
      --cv_folds 5 \
      --enable_preprocessing \
      --feature_selection combined \
      --n_top_features 300 \
      --device gpu \
      --seed 42
    print("\nFull CV analysis completed!")
else:
    print("Skipped full CV analysis.")

# Results Summary and Visualization

In [ ]:
# Check generated results
print("=== Generated Results ===")
!find ../experiment_results -name "*.json" -o -name "*.pkl" -o -name "*.png" | head -20

print("\n=== Legacy Results ===")
!find ../results -name "*.json" -o -name "*.pkl" -o -name "*.png" 2>/dev/null | head -10 || echo "No legacy results found"

print("\n=== Drive Storage ===")
!du -sh /content/drive/MyDrive/sgfa_experiments/ 2>/dev/null || echo "Drive path not accessible"

In [ ]:
# Generate experiment summary
summary_script = """
import sys, json, os
sys.path.append('.')

from experiments.framework import ExperimentRunner
from pathlib import Path

# Look for experiment results
results_dir = Path('../experiment_results')
if results_dir.exists():
    runner = ExperimentRunner()
    
    print("=== Experiment Summary ===")
    try:
        # Generate comprehensive report
        report = runner.generate_experiment_report()
        
        print(f"Total experiments: {len(report.get('experiments', []))}")
        print(f"Successful experiments: {report.get('summary', {}).get('successful', 0)}")
        print(f"Failed experiments: {report.get('summary', {}).get('failed', 0)}")
        
        # Save summary
        with open('../experiment_results/session_summary.json', 'w') as f:
            json.dump(report, f, indent=2, default=str)
            
        print("\nSummary saved to: ../experiment_results/session_summary.json")
        
    except Exception as e:
        print(f"Could not generate summary: {e}")
else:
    print("No experiment results directory found")
"""

with open("generate_summary.py", "w") as f:
    f.write(summary_script)

!py310cuda generate_summary.py

# Usage Notes

## Workflow Recommendations:

### For Quick Results (30-45 minutes):
1. Run **Data Validation** experiments
2. Run **Performance Benchmarks**
3. Run basic **Legacy Analysis** smoke tests

### For Comprehensive Results (60-90 minutes):
1. Run all **Experimental Framework** modules
2. Run **Advanced Analysis** pipelines
3. Generate comprehensive visualizations

### For Research Papers:
1. **Data Validation** → **Method Comparison** → **Clinical Validation**
2. Save optimal configs for transfer to remote GPU workstation
3. Use **Performance Benchmarks** for scaling analysis

## Key Improvements:
- **Modular experimental design** with standardized configs
- **GPU memory optimization** with adaptive batch sizing
- **Comprehensive validation** framework for reproducibility
- **Clinical validation** specific to Parkinson's disease research
- **Performance profiling** with detailed memory and timing analysis

## Results Location:
- **Experimental Framework**: `../experiment_results/`
- **Legacy Analysis**: `../results/`
- **Persistent Storage**: Google Drive (`sgfa_experiments/`)
